In [5]:
%load_ext autoreload
%autoreload 2

import os 
import sys

import pandas as pd

ROOT_PATH = os.path.dirname(os.getcwd())
sys.path.append(ROOT_PATH)

from src_.evals.run_model import run_model, build_model, load_saved_model
from src_.config import Config
from src_.utils.general import multi_target_train_test_split
from src_.evals.data_processing import get_and_process_data

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#### Load Data and Define Global Variables

In [6]:
DATA_PATH = os.path.join(ROOT_PATH, "data/210728_scrambles_for_unstructure_model.csv")
DATA2_PATH = os.path.join(ROOT_PATH, "data/210728_dmsv2_alldata.csv")

MODEL_TYPE = "convnet_1d"

In [7]:
X, y1, y2 = get_and_process_data(DATA_PATH)

X_train, X_test, y1_train, y1_test, y2_train, y2_test = \
        multi_target_train_test_split(X, y1, y2, return_val=False)


#### Run the model

In [10]:
model = load_saved_model(model_type=MODEL_TYPE,
                         save_dir = os.path.join(ROOT_PATH, "results/saved_weights"),
                         data_path=DATA_PATH)

OSError: Unable to open file (unable to open file: name = '/projects/p31346/karina/protease_stability/results/saved_weights/convnet_1d.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

#### Plot stability score

#### Global stability score

In [ ]:
from src.evals.stability_score import plot_stability_score_correlation

DATA2_PATH = os.path.join(ROOT_PATH, "data/210728_dmsv2_alldata.csv")
X, kT, kC = get_and_process_data(DATA2_PATH, return_as_df=True)
plot_stability_score_correlation(model, X.to_numpy(), np.array(kT), np.array(kC), sample = 10000)

#### Local stability score

In [11]:
data_folded = pd.read_csv(DATA2_PATH)
data_folded["pdb_code"] =  [name.split(".")[0] for name in data_folded.name]
grouped_indices = data_folded.groupby(by="pdb_code").indices

In [12]:
for i, (group_name, indices) in enumerate(grouped_indices.items()):
    if len(indices) > 10:
        X_, kT_, kC_ = X.loc[indices], kT.loc[indices], kC.loc[indices]
        kT_, kC_ = np.array(kT_), np.array(kC_)
        
        save_path = os.path.join(ROOT_PATH, "results/stability_scores/mutations/", f"{group_name}.png")

        plot_stability_score_correlation(model, X_, kT_, kC_, title=group_name, save_path = save_path)

    if i == 50:
        break

AttributeError: 'numpy.ndarray' object has no attribute 'loc'